In [29]:
try:
    import numpy as np
    from math import sqrt
    import math
    import scipy
    import random
    import qutip
except:
    !pip install qutip
    !pip install numpy
    !pip install scipy
    import numpy as np
    import scipy
    import qutip

In [31]:
# Find 
def find_entangled_matrices(n, m, num):
    entangled_matrices = []
    for i in range(num):
        matrix = qutip.rand_dm(n * m, dims=[[n, n], [m, m]])
        if np.all(qutip.partial_transpose(matrix, [0, 1]).eigenenergies()) > 0:
            entangled_matrices.append(matrix)
    return entangled_matrices

In [39]:
# PPT entangled state \in 3 \otimes 3
def rho(a):
    b = (1 + a) / 2
    c = (1 - a**2)**(1/2) / 2
    matrix = (1 / (8 * a + 1)) * np.array([
        [a, 0, 0, 0, a, 0, 0, 0, a],
        [0, a, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, a, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, a, 0, 0, 0, 0, 0],
        [a, 0, 0, 0, a, 0, 0, 0, a],
        [0, 0, 0, 0, 0, a, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, b, 0, c],
        [0, 0, 0, 0, 0, 0, 0, a, 0],
        [a, 0, 0, 0, a, 0, c, 0, b]
    ])
    return qutip.Qobj(matrix)

In [41]:
rho(0.5)

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[0.1        0.         0.         0.         0.1        0.
  0.         0.         0.1       ]
 [0.         0.1        0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.1        0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.1        0.         0.
  0.         0.         0.        ]
 [0.1        0.         0.         0.         0.1        0.
  0.         0.         0.1       ]
 [0.         0.         0.         0.         0.         0.1
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.15       0.         0.08660254]
 [0.         0.         0.         0.         0.         0.
  0.         0.1        0.        ]
 [0.1        0.         0.         0.         0.1        0.
  0.08660254 0.         0.15      ]]

In [97]:
"""
Entangled state \in 3 \otimes 3.

The following state \sigma(a) is:
    - separable for 2 <= a <= 3,
    - bound entangled for 3 < a <= 4,
    - free entangled for 4 < a <= 5.
"""

import qutip.tensor as tensor

def sigma(a):
    x = qutip.Qobj(np.array([1, 0, 0]))
    y = qutip.Qobj(np.array([0, 1, 0]))
    z = qutip.Qobj(np.array([0, 0, 1]))
    sigma_plus = 1/3 * (
        (tensor(x, y) * tensor(x.dag(), y.dag()))
        + (tensor(y, z) * tensor(y.dag(), z.dag()))
        + (tensor(z, x) * tensor(z.dag(), x.dag()))
    )
    sigma_minus = 1/3 * (
        (tensor(y, x) * tensor(y.dag(), x.dag()))
        + (tensor(z, y) * tensor(z.dag(), y.dag()))
        + (tensor(x, z) * tensor(x.dag(), z.dag()))
    )
    phi = (1 / (3 ** 0.5)) * (
        tensor(x, x) 
        + tensor(y, y) 
        + tensor(z, z)
    )
    return (2 / 7 * phi * phi.dag() 
            + a / 7 * sigma_plus 
            + (5 - a) / 7 * sigma_minus)

In [102]:
sigma(1)

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[0.0952381  0.         0.         0.         0.0952381  0.
  0.         0.         0.0952381 ]
 [0.         0.04761905 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.19047619 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.19047619 0.         0.
  0.         0.         0.        ]
 [0.0952381  0.         0.         0.         0.0952381  0.
  0.         0.         0.0952381 ]
 [0.         0.         0.         0.         0.         0.04761905
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.04761905 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.19047619 0.        ]
 [0.0952381  0.         0.         0.         0.0952381  0.
  0.         0.         0.0952381 ]]

In [49]:
def absolute_ppt(matrix):
    n, m = 3, 3
    l = [0]
    l.extend(list(np.sort(matrix.eigenenergies())[::-1]))
    l = np.array(l)
    
    L1 = np.array((
      [[2 * l[3*n], l[3*n-1] - l[1], l[3*n-3] - l[2]],
       [l[3*n-1] - l[1], 2 * l[3*n-2], l[3*n-4] - l[3]],
       [l[3*n-3] - l[2], l[3*n-4] - l[3], 2 * l[3*n-5]]]
    ))
    L2 = np.array((
      [[2 * l[3*n], l[3*n-1] - l[1], l[3*n-2] - l[2]],
       [l[3*n-1] - l[1], 2 * l[3*n-3], l[3*n-4] - l[3]],
       [l[3*n-2] - l[2], l[3*n-4] - l[3], 2 * l[3*n-5]]]
    ))
    return np.all(np.linalg.eigvals(L1) >= 0) and np.all(np.linalg.eigvals(L2) >= 0)

In [109]:
from tqdm import trange

it = 10000 
for i in trange(3 * it, 4 * it):
    if absolute_ppt(sigma(i/it)):
        print("Voila")
        break

100%|██████████| 10000/10000 [00:48<00:00, 204.17it/s]
